In [2]:
import pandas as pd
import numpy as np
import malaya
import pyLDAvis
from textblob import TextBlob

# import dataset
df = pd.read_csv('tweet_concat.csv')
#list(df)
df.shape

c:\users\user\anaconda3\envs\socionion\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""


(713, 31)

In [3]:
# features extraction
features = ['created_at','id','user','full_text']
data = df[features]
data.full_text = data.full_text.astype('str')
text = data.full_text
text_list = text.values.tolist() # model requires list as input

In [4]:
# language detection
mn_lang = malaya.language_detection.multinomial()
lang = mn_lang.predict_batch(text_list)

# add lang column
data['lang'] = lang
data.lang.value_counts() # ISSUE: model interprets malay as other

ENGLISH      331
OTHER        216
MALAY        160
INDONESIA      6
Name: lang, dtype: int64

In [5]:
# separate into english and malay
english = data[data['lang'] == 'ENGLISH']
english_text = english[['full_text']]
english_text_list = english_text.full_text.values.tolist() # model requires list as input

# other and indonesia mostly consist of malay based on observation
malay = data[data['lang'] != 'ENGLISH']
malay_text = malay[['full_text']]
malay_text_list = malay_text.full_text.values.tolist() # model requires list as input

In [10]:
# sentiment analysis and subjectivity analysis for english
english_sentiment = []
english_subjectivity = []

for tweet in english_text_list:
    blob = TextBlob(tweet)
    analysis = blob.sentiment
    sentiment = analysis[0]
    subject = analysis[1]
    # scale of -1 to 1
    if sentiment>0:
        english_sentiment.append('positive')
    elif sentiment<0:
        english_sentiment.append('negative')
    else:
        english_sentiment.append('neutral')
    # scale of 0 to 1
    if subject>0.6:
        english_subjectivity.append('subjective')
    elif subject<0.4:
        english_subjectivity.append('objective')
    else:
        english_subjectivity.append('mixed')
    
# add sentiment and subjectivity column
english['sentiment'] = english_sentiment
english['subjectivity'] = english_subjectivity
english.sentiment.value_counts()

positive    135
neutral     123
negative     73
Name: sentiment, dtype: int64

In [20]:
english.subjectivity.value_counts()

objective     172
mixed          88
subjective     71
Name: subjectivity, dtype: int64

In [ ]:
# sentiment analysis for malay
malay_sentiment_xgb = malaya.sentiment.xgb()
malay_sentiment = malay_sentiment_xgb.predict_batch(malay_text_list) # get_proba=True

# add sentiment column
malay['sentiment'] = malay_sentiment
malay.sentiment.value_counts()

In [ ]:
# topic modeling for english
stopwords_eng = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
lda = malaya.topic_model.lda(english_text_list,10,stemming=False,vectorizer='skip-gram',ngram=(1,4),skip=3,stop_words=stopwords_eng)

In [8]:
# return embeded visualization data
prepared_data = lda.visualize_topics(notebook_mode = False)

In [9]:
# save to html file for deployment
pyLDAvis.save_html(prepared_data, 'pylda.html')

In [18]:
# topic modeling for malay
lda2vec = malaya.topic_model.lda(malay_text_list,10,stemming=False,vectorizer='skip-gram',ngram=(1,4),skip=3)

In [19]:
lda2vec.visualize_topics(notebook_mode = False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.333686  0.064029       1        1  20.539265
4     -0.114100 -0.042708       2        1  13.326367
6     -0.102973  0.290873       3        1  11.867003
8     -0.013935 -0.068845       4        1  10.311939
7      0.067221 -0.039023       5        1   8.890253
9     -0.039171 -0.072080       6        1   8.596332
2     -0.054120  0.049312       7        1   8.559585
1     -0.005792 -0.081013       8        1   7.614653
0     -0.058109 -0.076562       9        1   7.320804
3     -0.012708 -0.023984      10        1   2.973799, topic_info=     Category        Freq                        Term       Total  loglift  \
5703  Default  203.000000                       unifi  203.000000  30.0000   
2873  Default   86.000000                          ni   86.000000  29.0000   
1791  Default   31.000000                          je   31.000000  28.0000   
2208  Default   29.000000                        line   29.000000  27.0000   
5498  Default   25.000000                          tu   25.000000  26.0000   
5934  Default   25.000000                    unifi ni   25.000000  25.0000   
2056  Default   25.000000                          la   25.000000  24.0000   
5881  Default   29.000000                unifi mobile   29.000000  23.0000   
2640  Default   26.000000                      mobile   26.000000  22.0000   
419   Default   18.000000                       bayar   18.000000  21.0000   
1740  Default   18.000000                    internet   18.000000  20.0000   
2947  Default   19.000000                      nombor   19.000000  19.0000   
4368  Default   23.000000                       rumah   23.000000  18.0000   
22    Default   18.000000                       akaun   18.000000  17.0000   
2365  Default   23.000000                        mbps   23.000000  16.0000   
1889  Default   13.000000                        kali   13.000000  15.0000   
5539  Default   25.000000                          tv   25.000000  14.0000   
3137  Default   12.000000                       pakai   12.000000  13.0000   
1942  Default   11.000000                        kena   11.000000  12.0000   
5511  Default   10.000000                       tukar   10.000000  11.0000   
1440  Default   24.000000                          hi   24.000000  10.0000   
2226  Default   17.000000                        live   17.000000   9.0000   
1076  Default   12.000000                        data   12.000000   8.0000   
2818  Default    9.000000                         nan    9.000000   7.0000   
2222  Default    9.000000                  line unifi    9.000000   6.0000   
2908  Default    9.000000                    ni unifi    9.000000   5.0000   
2948  Default    8.000000                nombor akaun    8.000000   4.0000   
3018  Default    9.000000              nombor telefon    9.000000   3.0000   
3666  Default   11.000000                        plan   11.000000   2.0000   
795   Default    9.000000                        call    9.000000   1.0000   
...       ...         ...                         ...         ...      ...   
879   Topic10    2.150409   channel berbayar tv baiki    2.852690   3.2327   
880   Topic10    2.150409  channel berbayar tv tonton    2.852690   3.2327   
881   Topic10    2.150409      channel berbayar tv tq    2.852690   3.2327   
4972  Topic10    2.150409   tablet tengok berbayar tv    2.852690   3.2327   
6167  Topic10    2.150409    versi berbayar tv tonton    2.852690   3.2327   
6166  Topic10    2.150409           versi berbayar tv    2.852690   3.2327   
445   Topic10    2.150409    berbayar tv tonton baiki    2.852690   3.2327   
5132  Topic10    2.150409      tengok tv tonton baiki    2.852690   3.2327   
6185  Topic10    2.150409      versi tengok tv tonton    2.852690   3.2327   
925   Topic10    2.150409        channel tv tonton tq    2.852690   3.2327   
924   Topic10    2.150409     cha

In [12]:
# ensemble
output = english.merge(malay, how='outer')
output

,created_at,id,user,full_text,lang,sentiment
0,Tue Apr 09 13:22:40 +0000 2019,1115605950998560768,"{'id': 136104635, 'id_str': '136104635', 'name...",Just wanna ask whether u prefer sleep on d pro...,ENGLISH,negative
1,Tue Apr 09 12:59:52 +0000 2019,1115600213484265477,"{'id': 1065994908148760578, 'id_str': '1065994...",hey unifi can you help me,ENGLISH,positive
2,Tue Apr 09 11:59:00 +0000 2019,1115584897567973382,"{'id': 175719741, 'id_str': '175719741', 'name...",Check out my Speedtest results 323 5 Mbps down...,ENGLISH,negative
3,Tue Apr 09 11:53:27 +0000 2019,1115583501628108800,"{'id': 94323843, 'id_str': '94323843', 'name':...",Join our postpaid plan OR refer 5 frens,ENGLISH,positive
4,Tue Apr 09 11:53:27 +0000 2019,1115583499300245504,"{'id': 94323843, 'id_str': '94323843', 'name':...",khabarbaik ED SHEERAN tickets for you YES you ...,ENGLISH,positive
5,Tue Apr 09 11:49:20 +0000 2019,1115582464322297857,"{'id': 94323843, 'id_str': '94323843', 'name':...",khabarbaik They are ready to win the RM10 000 ...,ENGLISH,positive
6,Tue Apr 09 09:34:03 +0000 2019,1115548417806102529,"{'id': 339368391, 'id_str': '339368391', 'name...",stupid unifi always down please don t be afrai...,ENGLISH,negative
7,Tue Apr 09 09:10:09 +0000 2019,1115542405065764864,"{'id': 44583755, 'id_str': '44583755', 'name':...",My wish is to earn just enough to pay for my c...,ENGLISH,negative
8,Tue Apr 09 06:16:28 +0000 2019,1115498696991334400,"{'id': 96706317, 'id_str': '96706317', 'name':...",The sendu laif of streamyx user Whenever it s ...,ENGLISH,positive
9,Tue Apr 09 05:35:51 +0000 2019,1115488475481018368,"{'id': 265909611, 'id_str': '265909611', 'name...",Malaysia Singapore agree to find amicable solu...,ENGLISH,positive
